In [205]:
import numpy as np
wordsList = np.load('wordsList.npy')
print('Loaded the word list!')
wordsList = wordsList.tolist() #Originally loaded as numpy array
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
wordVectors = np.load('wordVectors.npy')
print ('Loaded the word vectors!')

Loaded the word list!
Loaded the word vectors!


In [206]:
print (wordsList[0:10])
print (wordVectors[1])
print(len(wordsList))
print(wordVectors.shape)

['0', ',', '.', 'of', 'to', 'and', 'in', 'a', '"', "'s"]
[ 0.013441    0.23682    -0.16899     0.40950999  0.63812     0.47709
 -0.42851999 -0.55641001 -0.36399999 -0.23938     0.13000999 -0.063734
 -0.39574999 -0.48162001  0.23291001  0.090201   -0.13324     0.078639
 -0.41633999 -0.15428001  0.10068     0.48890999  0.31226    -0.1252
 -0.037512   -1.51789999  0.12612    -0.02442    -0.042961   -0.28351
  3.54159999 -0.11956    -0.014533   -0.1499      0.21864    -0.33412001
 -0.13872001  0.31806001  0.70358002  0.44858    -0.080262    0.63002998
  0.32111001 -0.46765     0.22786     0.36034    -0.37818    -0.56656998
  0.044691    0.30392   ]
400000
(400000, 50)


In [207]:
import tensorflow as tf
maxSeqLength = 200 #Maximum length of sentence
numDimensions = 300 #Dimensions for each word vector

In [208]:
baseballIndex = wordsList.index('jump')
print (baseballIndex)
print (wordVectors[baseballIndex])

3106
[-0.90742999  0.30886999  1.06040001  0.088596    0.30813     0.38332
 -0.43077999 -0.096476    0.51664001  0.186       0.28332001 -0.76336002
 -0.63599998  0.52166998  0.88101     0.1594      0.30474001 -0.31674001
 -1.57270002 -1.12030005  0.43566    -0.53811997 -0.37351999 -0.47691
  0.92426002 -0.99818999 -0.30322    -0.065498    0.49301001 -0.1758
  2.10019994  0.70359999  0.037695    0.78171998 -0.13428    -0.85176998
 -0.060642    0.47088     0.014336   -0.65079999 -0.59218001 -0.34836999
  0.34900999  0.012822    0.19881    -1.148       1.25660002 -0.35398999
  0.15462001 -0.007928  ]


In [209]:
firstSentence = np.zeros((maxSeqLength), dtype='int32')
firstSentence[0] = wordsList.index("i")
firstSentence[1] = wordsList.index("thought")
firstSentence[2] = wordsList.index("the")
firstSentence[3] = wordsList.index("movie")
firstSentence[4] = wordsList.index("was")
firstSentence[5] = wordsList.index("incredible")
firstSentence[6] = wordsList.index("and")
firstSentence[7] = wordsList.index("inspiring")
#firstSentence[8] and firstSentence[9] are going to be 0
print(firstSentence.shape)
print(firstSentence) #Shows the row index for each word

(200,)
[    41    804 201534   1005     15   7446      5  13767      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0

In [210]:
with tf.Session() as sess:
    print(tf.nn.embedding_lookup(wordVectors,firstSentence).eval().shape)

(200, 50)


In [211]:
# read stumple upon ever green content

In [212]:
import csv
import json
i=0
content = []
with open("stumbleupon/train.tsv") as f:
    tsvreader = csv.reader(f, delimiter="\t")
    for row in tsvreader:
        i = i  + 1
        if i < 2:
            continue
        #print (row[2])
        news = json.loads(row[2])
        #print (news)
        try:
            title = news["title"]
            if title == None:
                title = ""
            body = news["body"]
            if body == None:
                body = ""
        except KeyError:
            print ("skipping because of error")
        label = int(row[26])
        content.append((title,body,label))
        #print (title, label)

print (len(content))
#print (content[0:10])

skipping because of error
7395


In [213]:
# Removes punctuation, parentheses, question marks, etc., and leaves only alphanumeric characters
import re
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

In [214]:
i = 0
wordIndex = {}
for word in wordsList:
    wordIndex[word] = i
    i = i + 1

In [215]:
fileCounter = 0
ids = np.zeros((len(content), maxSeqLength), dtype='int32')
labels = np.zeros((len(content), 2), dtype='int32')
for (title,body,label) in content:
#     if fileCounter > 100:
#         break
    
    fullText = title + " " + body
    fullText = cleanSentences(fullText)
    #print (fullText)
    split = fullText.split()
    indexCounter = 0
    for word in split:
        #print (word)
        try:
            #print (word,indexCounter,  wordsList.index(word))
            #ids[fileCounter][indexCounter] = wordsList.index(word)
            ids[fileCounter][indexCounter] = wordIndex[word]
        except KeyError:
            ids[fileCounter][indexCounter] = 399999 #Vector for unkown words
        indexCounter = indexCounter + 1
        #print (indexCounter)
        if indexCounter >= maxSeqLength:
            break
    if label == 1:
        labels[fileCounter,:] = [1,0]
    else:
        labels[fileCounter,:] = [0,1]

    fileCounter = fileCounter + 1 

In [216]:
print (ids[500:510,:], labels[500:510,:])
print (len(ids))

[[    55   4719   8628 ...,   6988      5      7]
 [    96  10267      5 ...,    101   1547     12]
 [  5578   1163  10817 ...,   3634    314    409]
 ..., 
 [    66      3     37 ...,     39    390    841]
 [ 25663   3305    719 ...,  31929  24309 201534]
 [  8628  19829   5015 ...,      0      0      0]] [[0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]]
7395


In [217]:

train_size = 5000
test_size = len(labels) - train_size
print ("test_size",test_size)
train_ids = ids[0:train_size,:]
train_labels = labels[0:train_size,:]
test_ids = ids[train_size+1:,:]
test_labels = labels[train_size+1:,:]
#print (train_labels)
#print (test_labels)

print(len(train_ids))

test_size 2395
5000


In [218]:
batchSize = 24
lstmUnits = 64
numClasses = 2
iterations = 40000

In [219]:
from random import randint

def getBatch(train, labels):
    batch_labels = []
    arr = np.zeros([batchSize, maxSeqLength])
    size = len(train)
    #print (size)
    for i in range(batchSize):
        num = randint(1,size)
        labels_list = labels[num-1:num][0,:].tolist()
        batch_labels.append(labels_list)
        arr[i] = train[num-1:num]
    return arr, batch_labels

In [220]:
ret = getBatch(train_ids,train_labels)
#print(ret[0],ret[1])
ret = getBatch(test_ids,test_labels)
#print(ret[0],ret[1])


In [221]:
import tensorflow as tf
tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])

In [222]:
data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)

In [223]:
lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.75)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

In [224]:
weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

In [225]:
correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

In [226]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(loss)

In [227]:
import datetime

tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

In [228]:

iterations = 10010

sess = tf.InteractiveSession()
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())

for i in range(iterations):
   #Next Batch of reviews
    nextBatch, nextBatchLabels = getBatch(train_ids,train_labels)
    sess.run(optimizer, {input_data: nextBatch, labels: nextBatchLabels})
   
   #Write summary to Tensorboard
    if (i % 50 == 0):
        summary = sess.run(merged, {input_data: nextBatch, labels: nextBatchLabels})
        writer.add_summary(summary, i)

   #Save the network every 10,000 training iterations
    if (i % 10000 == 0 and i != 0):
        save_path = saver.save(sess, "models/pretrained_lstm.ckpt", global_step=i)
        print("saved to %s" % save_path)
writer.close()

saved to models/pretrained_lstm.ckpt-10000


In [135]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint('models'))

INFO:tensorflow:Restoring parameters from models/pretrained_lstm.ckpt-40000


NotFoundError: Key Variable_1/Adam_1 not found in checkpoint
	 [[Node: save_1/RestoreV2_3 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_1/Const_0_0, save_1/RestoreV2_3/tensor_names, save_1/RestoreV2_3/shape_and_slices)]]

Caused by op 'save_1/RestoreV2_3', defined at:
  File "/Users/nahmad/anaconda2/envs/py36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/nahmad/anaconda2/envs/py36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/nahmad/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/nahmad/anaconda2/envs/py36/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/nahmad/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().

In [230]:
iterations = 1000
accur = []
for i in range(iterations):
    nextBatch, nextBatchLabels = getBatch(test_ids,test_labels)
    a = (sess.run(accuracy, {input_data: nextBatch, labels: nextBatchLabels})) * 100
    accur.append(a)
    print("Accuracy for this batch:", a)

sum = 0
for s in accur:
    sum += s
    
print ("Average Accuracy ", sum/len(accur))

Accuracy for this batch: 87.5
Accuracy for this batch: 62.5
Accuracy for this batch: 66.6666686535
Accuracy for this batch: 91.6666686535
Accuracy for this batch: 62.5
Accuracy for this batch: 70.8333313465
Accuracy for this batch: 70.8333313465
Accuracy for this batch: 70.8333313465
Accuracy for this batch: 66.6666686535
Accuracy for this batch: 87.5
Accuracy for this batch: 66.6666686535
Accuracy for this batch: 70.8333313465
Accuracy for this batch: 66.6666686535
Accuracy for this batch: 70.8333313465
Accuracy for this batch: 75.0
Accuracy for this batch: 75.0
Accuracy for this batch: 75.0
Accuracy for this batch: 75.0
Accuracy for this batch: 79.1666686535
Accuracy for this batch: 79.1666686535
Accuracy for this batch: 70.8333313465
Accuracy for this batch: 66.6666686535
Accuracy for this batch: 75.0
Accuracy for this batch: 79.1666686535
Accuracy for this batch: 70.8333313465
Accuracy for this batch: 79.1666686535
Accuracy for this batch: 66.6666686535
Accuracy for this batch: 83.

Accuracy for this batch: 66.6666686535
Accuracy for this batch: 70.8333313465
Accuracy for this batch: 70.8333313465
Accuracy for this batch: 83.3333313465
Accuracy for this batch: 58.3333313465
Accuracy for this batch: 91.6666686535
Accuracy for this batch: 79.1666686535
Accuracy for this batch: 79.1666686535
Accuracy for this batch: 91.6666686535
Accuracy for this batch: 83.3333313465
Accuracy for this batch: 75.0
Accuracy for this batch: 70.8333313465
Accuracy for this batch: 79.1666686535
Accuracy for this batch: 87.5
Accuracy for this batch: 66.6666686535
Accuracy for this batch: 66.6666686535
Accuracy for this batch: 83.3333313465
Accuracy for this batch: 75.0
Accuracy for this batch: 83.3333313465
Accuracy for this batch: 79.1666686535
Accuracy for this batch: 62.5
Accuracy for this batch: 62.5
Accuracy for this batch: 75.0
Accuracy for this batch: 62.5
Accuracy for this batch: 75.0
Accuracy for this batch: 87.5
Accuracy for this batch: 75.0
Accuracy for this batch: 75.0
Accurac

Accuracy for this batch: 83.3333313465
Accuracy for this batch: 70.8333313465
Accuracy for this batch: 79.1666686535
Accuracy for this batch: 87.5
Accuracy for this batch: 79.1666686535
Accuracy for this batch: 79.1666686535
Accuracy for this batch: 95.8333313465
Accuracy for this batch: 70.8333313465
Accuracy for this batch: 58.3333313465
Accuracy for this batch: 87.5
Accuracy for this batch: 83.3333313465
Accuracy for this batch: 70.8333313465
Accuracy for this batch: 87.5
Accuracy for this batch: 75.0
Accuracy for this batch: 79.1666686535
Accuracy for this batch: 83.3333313465
Accuracy for this batch: 75.0
Accuracy for this batch: 75.0
Accuracy for this batch: 70.8333313465
Accuracy for this batch: 79.1666686535
Accuracy for this batch: 75.0
Accuracy for this batch: 58.3333313465
Accuracy for this batch: 75.0
Accuracy for this batch: 66.6666686535
Accuracy for this batch: 79.1666686535
Accuracy for this batch: 79.1666686535
Accuracy for this batch: 75.0
Accuracy for this batch: 75.

Accuracy for this batch: 70.8333313465
Accuracy for this batch: 79.1666686535
Accuracy for this batch: 83.3333313465
Accuracy for this batch: 87.5
Accuracy for this batch: 87.5
Accuracy for this batch: 79.1666686535
Accuracy for this batch: 70.8333313465
Accuracy for this batch: 70.8333313465
Accuracy for this batch: 87.5
Accuracy for this batch: 83.3333313465
Accuracy for this batch: 75.0
Accuracy for this batch: 75.0
Accuracy for this batch: 75.0
Accuracy for this batch: 66.6666686535
Accuracy for this batch: 62.5
Accuracy for this batch: 75.0
Accuracy for this batch: 70.8333313465
Accuracy for this batch: 66.6666686535
Accuracy for this batch: 75.0
Accuracy for this batch: 75.0
Accuracy for this batch: 83.3333313465
Accuracy for this batch: 87.5
Accuracy for this batch: 70.8333313465
Accuracy for this batch: 79.1666686535
Accuracy for this batch: 70.8333313465
Accuracy for this batch: 87.5
Accuracy for this batch: 70.8333313465
Accuracy for this batch: 95.8333313465
Accuracy for thi

Accuracy for this batch: 83.3333313465
Accuracy for this batch: 83.3333313465
Accuracy for this batch: 75.0
Accuracy for this batch: 75.0
Accuracy for this batch: 91.6666686535
Accuracy for this batch: 70.8333313465
Accuracy for this batch: 62.5
Accuracy for this batch: 75.0
Accuracy for this batch: 70.8333313465
Accuracy for this batch: 75.0
Accuracy for this batch: 79.1666686535
Accuracy for this batch: 95.8333313465
Accuracy for this batch: 75.0
Accuracy for this batch: 70.8333313465
Accuracy for this batch: 79.1666686535
Accuracy for this batch: 75.0
Accuracy for this batch: 83.3333313465
Accuracy for this batch: 79.1666686535
Accuracy for this batch: 79.1666686535
Accuracy for this batch: 79.1666686535
Accuracy for this batch: 79.1666686535
Accuracy for this batch: 62.5
Accuracy for this batch: 75.0
Accuracy for this batch: 87.5
Accuracy for this batch: 83.3333313465
Accuracy for this batch: 91.6666686535
Accuracy for this batch: 83.3333313465
Accuracy for this batch: 62.5
Accurac